<a href="https://colab.research.google.com/github/Darth-Kronos/Spectrum-Sensing/blob/master/DNN_FM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Library  { form-width: "30%" }
import tensorflow.keras as keras
from scipy.stats import entropy
import tensorflow as tf
import numpy as np
import time
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt 
import pandas as pd 
from scipy.fft import fft
from scipy.stats import gennorm
import numpy as np
from scipy.special import gamma
import math
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense,LSTM
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from scipy.signal import savgol_filter
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.utils import np_utils
!pip install -q -U tensorflow-addons
!pip install keras-tcn
drive.mount('/content/drive')
# from utils.utils import save_logs
# from utils.utils import calculate_metrics

In [ ]:
#@title DATA  { form-width: "30%" }
def find_energy(signal, snr):
  Y = fft(signal)
  abs_Y = abs(Y)**2
  r = np.mean(abs_Y)
  return 2*snr*np.sum(r)

def find_gp(signal):
  return np.exp(np.mean(np.log(abs(np.array(signal)))))

def find_lp(signal,p):
  lp = np.mean(abs(signal)**p)
  return lp 

def normalizeData(raw_data):
  # print(np.var(raw_data))
  data = (raw_data - np.mean(raw_data))/np.sqrt(np.var(raw_data))
  return data

def find_de(signal):
    pdf,_ = np.histogram(signal)
    pdf = pdf/len(signal)
    return entropy(pdf,base=2)

batch = 50
df = pd.read_csv('Dataset/Dataset3.csv')
raw_data = df.iloc[:,0]
raw_data = raw_data.to_numpy()
signal = normalizeData(raw_data[0:50000])
lenSample = len(signal)
# beta = 1.5
TIME_PERIODS = 1
def createTrainTest(data,Features):
  x = data[:,0:Features]
  y = data[:,-1]
  return x,y

def createFeature(beta,snrDB):
  var = 1
  p=0.5

  snr = 10**(snrDB/10)
  featuresMatrix = np.zeros(shape=(2000,5))
  dfNoise = pd.read_csv('Dataset/FMNoise.csv',header=None,index_col=0)
  noise = normalizeData(dfNoise.values.flatten())
  
  for i in range(0,lenSample,batch):
    if i+batch <= lenSample:

      noise_fm = np.random.choice(noise,batch)
    
      h1 =  noise_fm +  np.sqrt(snr)*signal[i:i+batch]
      h0 = np.random.choice(noise,batch)
      # features for h1
      energy = find_energy(h1,snr)
      de = find_de1(h1)
      gp = find_gp(h1)
      lp = find_lp(h1,p)
      featuresMatrix[i//batch] = [gp,de,lp,energy,1]
      
      # features for h0
      energy = find_energy(h0,snr)
      de = find_de1(h0)
      gp = find_gp(h0)
      lp = find_lp(h0,p)
      featuresMatrix[(lenSample+i)//batch] = [gp,de,lp,energy,0]
      

  featuresMatrix[:,0] = featuresMatrix[:,0]/max(featuresMatrix[:,0])
  featuresMatrix[:,1] = featuresMatrix[:,1]/max(featuresMatrix[:,1])
  featuresMatrix[:,2] = featuresMatrix[:,2]/max(featuresMatrix[:,2])
  featuresMatrix[:,3] = featuresMatrix[:,3]/max(featuresMatrix[:,3])

  np.random.shuffle(featuresMatrix)

  return featuresMatrix
  
def findPd(pf,pd):
  temp = np.where((pf<=0.1) & (pf>0.0))[0]
  if temp.size>0 and pd[temp[-1]] != 0.0:
    return pd[temp[-1]]
  else:
    return 1.0

In [ ]:
snrDB = np.flip(np.arange(-20,0.5,0.5))

beta = 1

pd_cnn = np.ones(len(snrDB))
pd_mlp = np.ones(len(snrDB))
pd_fcn = np.ones(len(snrDB))
pd_rnet =np.ones(len(snrDB))
pd_lstm =np.ones(len(snrDB))
pd_tcn = np.ones(len(snrDB))

num_time_periods = 1
num_classes = 1
num_sensors=4

In [ ]:
#@title MLP  { form-width: "30%" }

model_mlp = Sequential()
model_mlp.add(keras.Input(shape=(num_sensors,)))

model_mlp.add(Dense(100,activation='relu'))
model_mlp.add(Dense(100,activation='relu'))
model_mlp.add(Dense(100,activation='relu'))
model_mlp.add(Dense(1,activation='sigmoid'))

In [ ]:
#@title CNN  { form-width: "30%" }
model_cnn = Sequential()
model_cnn.add(Reshape((1, num_sensors), input_shape=(num_sensors,)))
model_cnn.add(Conv1D(50, 1, activation='relu', input_shape=(1,num_sensors),padding='same'))
model_cnn.add(Conv1D(50, 1, activation='relu'))
model_cnn.add(MaxPooling1D(1,2))
model_cnn.add(Dense(50,activation='relu'))
model_cnn.add(Dense(50,activation='relu'))
model_cnn.add(GlobalAveragePooling1D())
model_cnn.add(Dense(1, activation='sigmoid'))


In [ ]:
#@title RESNET  { form-width: "30%" }
n_feature_maps = 50

input_layer = keras.layers.Input(num_sensors,)

# BLOCK 1
reshape = keras.layers.Reshape((1,num_sensors), input_shape=(num_sensors,))(input_layer)
conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(reshape)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(reshape)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_1 = keras.layers.add([shortcut_y, conv_z])
output_block_1 = keras.layers.Activation('relu')(output_block_1)

# BLOCK 2

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_2 = keras.layers.add([shortcut_y, conv_z])
output_block_2 = keras.layers.Activation('relu')(output_block_2)

# BLOCK 3

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# no need to expand channels because they are equal
shortcut_y = keras.layers.BatchNormalization()(output_block_2)

output_block_3 = keras.layers.add([shortcut_y, conv_z])
output_block_3 = keras.layers.Activation('relu')(output_block_3)

# FINAL

gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

output_layer = keras.layers.Dense(1, activation='sigmoid')(gap_layer)

model_rnet = keras.models.Model(inputs=input_layer, outputs=output_layer)

# model_m.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(model_m, to_file='model_plot.png', show_shapes=True, show_layer_names=True)



In [ ]:
#@title FCN  { form-width: "30%" }
input_layer = keras.layers.Input(num_sensors,)
reshape = keras.layers.Reshape((1, num_sensors), input_shape=(num_sensors,))(input_layer)
conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(reshape)
conv1 = keras.layers.BatchNormalization()(conv1)
conv1 = keras.layers.Activation(activation='relu')(conv1)

conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
conv2 = keras.layers.BatchNormalization()(conv2)
conv2 = keras.layers.Activation('relu')(conv2)

conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
conv3 = keras.layers.BatchNormalization()(conv3)
conv3 = keras.layers.Activation('relu')(conv3)

gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

output_layer = keras.layers.Dense(1, activation='sigmoid')(gap_layer)

model_fcn = keras.models.Model(inputs=input_layer, outputs=output_layer)


In [ ]:
#@title LSTM  { form-width: "30%" }

from keras.regularizers import l2
# Import Keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time


N = num_sensors                 # number of features
EPOCH = 50                           # number of epochs
LR = 5e-2                            # learning rate of the gradient descent
LAMBD = 3e-2                         # lambda in L2 regularizaion
DP = 0.0                             # dropout rate
RDP = 0.0                            # recurrent dropout rate

model_lstm = Sequential()
model_lstm.add(Reshape((1,num_sensors), input_shape=(num_sensors,)))
model_lstm.add(LSTM(input_shape=(1,num_sensors), units=8,
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model_lstm.add(BatchNormalization())
model_lstm.add(LSTM(units=8,
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model_lstm.add(BatchNormalization())
model_lstm.add(LSTM(units=8,
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=False, return_state=False,
               stateful=False, unroll=False
              ))
model_lstm.add(BatchNormalization())
model_lstm.add(Dense(units=1, activation='sigmoid'))

model_lstm.summary()

In [ ]:
#@title encoder  { form-width: "30%" }
# import tensorflow.keras as keras
# import tensorflow as tf
# import tensorflow_addons as tfa
# import numpy as np
# import time

# input_layer = keras.layers.Input(3,)
# reshape = keras.layers.Reshape((1, 3), input_shape=(3,))(input_layer)
# # conv block -1
# conv1 = keras.layers.Conv1D(filters=128,kernel_size=5,strides=1,padding='same')(reshape)
# conv1 = tfa.layers.InstanceNormalization()(conv1)
# conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
# conv1 = keras.layers.Dropout(rate=0.2)(conv1)
# conv1 = keras.layers.MaxPooling1D(pool_size=1)(conv1)
# # conv block -2
# conv2 = keras.layers.Conv1D(filters=256,kernel_size=11,strides=1,padding='same')(conv1)
# conv2 = tfa.layers.InstanceNormalization()(conv2)
# conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
# conv2 = keras.layers.Dropout(rate=0.2)(conv2)
# conv2 = keras.layers.MaxPooling1D(pool_size=1)(conv2)
# # conv block -3
# conv3 = keras.layers.Conv1D(filters=512,kernel_size=21,strides=1,padding='same')(conv2)
# conv3 = tfa.layers.InstanceNormalization()(conv3)
# conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
# conv3 = keras.layers.Dropout(rate=0.2)(conv3)
# # split for attention
# attention_data = keras.layers.Lambda(lambda x: x[:,:,:256])(conv3)
# attention_softmax = keras.layers.Lambda(lambda x: x[:,:,256:])(conv3)
# # attention mechanism
# attention_softmax = keras.layers.Softmax()(attention_softmax)
# multiply_layer = keras.layers.Multiply()([attention_softmax,attention_data])
# # last layer
# dense_layer = keras.layers.Dense(units=256,activation='sigmoid')(multiply_layer)
# dense_layer = tfa.layers.InstanceNormalization()(dense_layer)
# # output layer
# flatten_layer = keras.layers.Flatten()(dense_layer)
# output_layer = keras.layers.Dense(units=1,activation='sigmoid')(flatten_layer)

# model_m = keras.models.Model(inputs=input_layer, outputs=output_layer)
# model_m.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(model_m, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [ ]:
#@title TCN
from tcn import compiled_tcn
from tcn import TCN, tcn_full_summary

model_tcn = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=1,
                         nb_filters=20,
                         kernel_size=6,
                         dilations=[2 ** i for i in range(9)],
                         nb_stacks=1,
                         max_len=num_sensors,
                         use_weight_norm=True,
                         use_skip_connections=True)
model_tcn.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(model_m, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#@title Model train/test  { form-width: "30%" }
def trainingModel(model,tcn=False):
  if tcn:
    model.compile(
    loss='sparse_categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

    # Hyper-parameters
    BATCH_SIZE = 200
    EPOCHS = 5

    # Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
    history = model.fit(x_train,
                          y_train,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,

                          validation_split=0.2,
                          verbose=0
                          )
    y_pred = model.predict(x_test)

    y_pred2=[]
    for i in y_pred:
      # print(i[0])
      if(i[0]>i[1]):
        y_pred2.append(i[1])
      else:
        y_pred2.append(i[1])
    y_test2=[]
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test,y_pred2)
    return fpr_keras,tpr_keras

  # callbacks_list = [
  #     keras.callbacks.ModelCheckpoint(
  #         filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
  #         monitor='val_loss', save_best_only=True),
  #     keras.callbacks.EarlyStopping(monitor='accuracy', patience=1)
  # ]

  model.compile(
      loss='binary_crossentropy',
                  optimizer='adam', metrics=['binary_accuracy'])

  # Hyper-parameters
  BATCH_SIZE = 10
  EPOCHS = 10

  # Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
  history = model.fit(x_train,
                        y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                
                        validation_split=0.2,
                        verbose=0
                        )
  accuracy_results = model.evaluate(x_test, y_test)
  print("Accuracy :",accuracy_results)

  y_pred = model.predict(x_test)
  
  fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test,y_pred)
  
  return fpr_keras,tpr_keras

In [ ]:
#@title Run  { form-width: "30%" }

for i in range(len(snrDB)):
  featuresMatrix=createFeature(beta,snrDB[i])
  
  df_train,df_test = train_test_split(featuresMatrix, test_size=0.2, random_state=42)
  x_train, y_train = createTrainTest(df_train,num_sensors)
  x_test, y_test = createTrainTest(df_test,num_sensors)
  
  print(snrDB[i])
  input_shape = (num_time_periods*num_sensors)
  x_train = x_train.reshape(x_train.shape[0], input_shape)
  x_train = x_train.astype("float32")
  y_train = y_train.astype("float32")

  x_test = x_test.reshape(x_test.shape[0], input_shape)

  x_test = x_test.astype("float32")
  y_test = y_test.astype("float32")

  fpr_cnn,tpr_cnn = trainingModel(model_cnn)
  pd_cnn[i]=findPd(fpr_cnn,tpr_cnn)

  fpr_fcn,tpr_fcn = trainingModel(model_fcn)
  pd_fcn[i]=findPd(fpr_fcn,tpr_fcn)

  fpr_lstm,tpr_lstm = trainingModel(model_lstm)
  pd_lstm[i]=(findPd(fpr_lstm,tpr_lstm))

  fpr_mlp,tpr_mlp = trainingModel(model_mlp)
  pd_mlp[i]=(findPd(fpr_mlp,tpr_mlp))

  fpr_rnet,tpr_rnet = trainingModel(model_rnet)
  pd_rnet[i]=(findPd(fpr_rnet,tpr_rnet))

  fpr_tcn,tpr_tcn = trainingModel(model_tcn,tcn=True)
  pd_tcn[i]=(findPd(fpr_tcn,tpr_tcn))

pd_cnn = savgol_filter(pd_cnn, 21, 2)
pd_mlp = savgol_filter(pd_mlp, 21, 2)
pd_lstm = savgol_filter(pd_lstm, 21, 2)
pd_rnet = savgol_filter(pd_rnet, 21, 2)
pd_tcn = savgol_filter(pd_tcn, 21, 2)
pd_fcn = savgol_filter(pd_fcn, 21, 2)